In [25]:
import requests
import json
import pandas as pd
from datetime import datetime

webpage = requests.get("https://wusa.ca/events/")
jsonscript =str(webpage.content)
isolatedinformation=jsonscript.split('<script type="application/ld+json">')[1].split("</script>")[0][4:-4].encode("utf16", errors="surrogatepass").decode("utf16").encode().decode('unicode_escape')

json = json.loads(isolatedinformation.replace('&lt;p&gt;',"").replace("[&hellip;]&lt;/p&gt;\\\\n",""))

postsDf ={}
postscolumns = ['account','date','caption',"display_photo",'event_details']
postsDf = pd.DataFrame(columns = postscolumns)

In [26]:
import time

time_now = int(time.time())
for event in json:
    if int(datetime.fromisoformat(event["startDate"]).timestamp()) >= time_now:
        try: 
            location = event["location"]["address"]["streetAddress"]
        except Exception as err:
            print(str(err))
            location = None
        
        new_row = pd.DataFrame({
                "account": ["WUSA"],
                "date": [time_now],
                "caption": str(event["description"])+" [For More Information, Click View Post] ",
                "display_photo": event["image"],
                "url": [event['url']],
                "likes": [0],
                "event_details": [{
                    "is_event": True,
                    "event_name": event["name"],
                    "event_description": str(event["description"])+" ... ",
                    "categories": [["SOCIAL"]],
                    "start_time": int(datetime.fromisoformat(event["startDate"]).timestamp()+ 7200),
                    "end_time": int(datetime.fromisoformat(event["endDate"]).timestamp()+ 7200),
                    "location": location,
                }]
        })

        postsDf = pd.concat([postsDf, new_row])

print(postsDf)
postsDf.to_csv(path_or_buf="WUSA events")

'location'
'streetAddress'
'streetAddress'
'streetAddress'
  account        date                                            caption  \
0    WUSA  1725566062  CIBC x WUSA x UWaterloo presents Dillon Franci...   
0    WUSA  1725566062  Welcome Week 🍂 Get ready to FALL back into fun...   
0    WUSA  1725566062  Rise and shine, Warriors! WUSA’s inviting you ...   
0    WUSA  1725566062  Esports Gaming Lounge Game on, Warriors! Brace...   
0    WUSA  1725566062  😍 A Welcome Week favourite is back! Are you re...   
0    WUSA  1725566062  Step right up! The WUSA Carnival is back for T...   
0    WUSA  1725566062  To give you a warm welcome back to campus, we ...   
0    WUSA  1725566062  Step right up! The WUSA Carnival is back for T...   
0    WUSA  1725566062  Don\'t miss out on the ultimate bowling experi...   
0    WUSA  1725566062  Esports Gaming Lounge Game on, Warriors! Brace...   
0    WUSA  1725566062  The Campus Life Fair is back!   Join us on Wed...   
0    WUSA  1725566062  Thrift

In [27]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# Remove all documents with these IDs
result = collection.delete_many({'account': "WUSA"})

# Print the number of documents deleted
print(f"Documents removed: {result.deleted_count}")


Documents removed: 21


In [28]:
from typing import List
from together import Together
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

insertObjectIds = []

for index, row in postsDf.iterrows():
    info = f'"id": "{int(index)}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|*'
    embbededtext = ',\n'.join(x for x in info.replace('\n','\\n').split('|*')) 

    row_dict = {}
    for column in postsDf.columns.tolist():
        row_dict[column] = row[column]
    row_dict["embedded"] = generate_embedding([embbededtext], embedding_model_string)  
    result = collection.insert_one(row_dict)
    print(f"Inserted document ID: {result.inserted_id}")
    insertObjectIds.append(result.inserted_id)
    time.sleep(0.5)


Inserted document ID: 66da0c7e794b14a49ee0615c
Inserted document ID: 66da0c7f794b14a49ee0615d
Inserted document ID: 66da0c81794b14a49ee0615e
Inserted document ID: 66da0c82794b14a49ee0615f
Inserted document ID: 66da0c85794b14a49ee06160
Inserted document ID: 66da0c85794b14a49ee06161
Inserted document ID: 66da0c86794b14a49ee06162
Inserted document ID: 66da0c88794b14a49ee06163
Inserted document ID: 66da0c89794b14a49ee06164
Inserted document ID: 66da0c8a794b14a49ee06165
Inserted document ID: 66da0c8b794b14a49ee06166
Inserted document ID: 66da0c8c794b14a49ee06167
Inserted document ID: 66da0c8e794b14a49ee06168
Inserted document ID: 66da0c8f794b14a49ee06169
Inserted document ID: 66da0c91794b14a49ee0616a
Inserted document ID: 66da0c92794b14a49ee0616b
Inserted document ID: 66da0c94794b14a49ee0616c
Inserted document ID: 66da0c95794b14a49ee0616d
Inserted document ID: 66da0c96794b14a49ee0616e
Inserted document ID: 66da0c98794b14a49ee0616f
Inserted document ID: 66da0c98794b14a49ee06170
